In [0]:
#Rachael's/PID Eq-- questions about whether this actually covers the full dynamics

#might have to update these values?
alpha = 10e-2
gamma = 7.535e-5

def regulation(df, alpha=10e-2, gamma=7.535e-5):
  ## calculate the prediction with current regulation rules
  ## from Rachael's report, eq (1)
  beta=[0]
  ER = df["B:IMINER"].astype('float') #error
  _MIN = df["B_VIMIN"].astype('float') #setting
  for i in range(len(_MIN)):
      if i>0:
          beta_t = beta[-1] + gamma*ER[i]
          beta.append(beta_t)
  MIN_pred = _MIN - alpha * ER - beta #predict the next, shiftting happens in the plotting
  return MIN_pred

In [0]:
def scale(df, var_list = ['B:VIMIN', 'B:IMINER', 'B_VIMIN', 'B:LINFRQ', 'B:ACMNIG', 'B:ACMNPG', 'I:IB', 'I:MDAT40']):
  scale_dict = {}

  for var in var_list:
  #for var in ['B:ACMNIG', 'B:ACMNPG']:
    our_data2 = df
    trace = our_data2[var].astype('float32')
    data = np.array(trace)
    median = np.median(data)
    upper_quartile = np.percentile(data, 75)
    lower_quartile = np.percentile(data, 25)

    iqr = upper_quartile - lower_quartile
    lower_whisker = data[data>=lower_quartile-1.5*iqr].min()
    upper_whisker = data[data<=upper_quartile+1.5*iqr].max()

    if var == 'B:ACMNPG':
      median = 9.1968
      upper_whisker = median + 0.75*0.674
      lower_whisker = median - 0.75*0.674
    elif var == 'B:ACMNIG':
      upper_whisker = 0.75350088
      lower_whisker = 0

    print(f'Variable {var} with upper {upper_whisker} and lower {lower_whisker}')

    ranged = upper_whisker - lower_whisker
    #(value − median) / (upper - lower)

    our_data2[var] = 1/ranged*(data - median)

    scale_dict[str(var)] = {"median": median, "range": ranged}
  # print(scale_dict)

  return(our_data2, scale_dict)

#need this for all the plotting functions
def unscale(var_name, tseries, scale_dict):
  from_model = np.asarray(tseries)
  update = from_model*scale_dict[str(var_name)]["range"] + scale_dict[str(var_name)]["median"]
  return(update)

In [0]:
# we moved away from using these scalers since it's difficult to implement them in C++ on the board
#from sklearn.preprocessing import MinMaxScaler, RobustScaler

def create_dataset(dataset, look_back=1,look_forward=1):
    X, Y = [], []
    offset = look_back+look_forward
    for i in range(len(dataset)-(offset+1)):
        xx = dataset[i:(i+look_back), 0]
        yy = dataset[(i + look_back):(i + offset), 0]
        X.append(xx)
        Y.append(yy)
    return np.array(X), np.array(Y)

def get_dataset(data, variable='B:VIMIN'):
    df = data
    dataset = df[variable].values #numpy.ndarray
    dataset = dataset.astype('float32')
    dataset = np.reshape(dataset, (-1, 1))
#     scaler = MinMaxScaler(feature_range=(0, 1)) #i'm going to keep saving the minmax scaler just so i don't have to update a lot of things
    #2/28: jokes not changing this yet... need to know how malachi wants me to implement the inverse scaling
    #scaler = RobustScaler()
#     dataset = scaler.fit_transform(dataset)

    ## TODO: Fix
    #print(len(dataset))
    train_size = int(len(dataset) * 0.70)
    #print(train_size)
    test_size = len(dataset) - train_size
    #print(test_size)

    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

    X_train, Y_train = create_dataset(train, look_back,look_forward)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    Y_train = np.reshape(Y_train, (Y_train.shape[0],  Y_train.shape[1]))
    #print(X_train.shape)
    #print(Y_train.shape)
    
    X_test, Y_test = create_dataset(test, look_back,look_forward)
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
    Y_test = np.reshape(Y_test, (Y_test.shape[0],  Y_test.shape[1]))
    #print(X_test.shape)
    #print(Y_test.shape)
#     return scaler, X_train, Y_train, X_test, Y_test
    return X_train, Y_train, X_test, Y_test
  #reminder that we've changed the scaler so there are different indices now: return X_train, Y_train, X_test, Y_test but no scaler

In [0]:
def loss_plot(fold, histories, model, BoX_test, BoY_test):
  
  if fold == 4:
    print(histories[0].history['loss'])
    print(histories[1].history['loss'])
    print(histories[2].history['loss'])
    print(histories[3].history['loss'])
    print(histories[4].history['loss'])
    
    loss_trace = []
    vloss_trace = []
    
    for k in range(5):
        #print(k)
        #print(k)
        #print(histories[k])
        fold_histories = np.array(histories[k].history['loss'])
        #print(fold_histories.shape)
        loss_trace.append(fold_histories)
        vloss_trace.append(np.array(histories[k].history['val_loss']))

        full_loss_trace = np.concatenate(loss_trace)
        full_vloss_trace = np.concatenate(vloss_trace)
        #print(full_loss_trace.shape)
        
    plt.figure(figsize=(12,10))
    plt.plot(full_loss_trace, label='loss')
    plt.plot(full_vloss_trace, label='val_loss')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(loc='upper right')
    plt.yscale('log')

    plt.savefig('loss_{}.png'.format(fold))
    mlflow.log_artifact('loss_{}.png'.format(fold))

    dump(full_loss_trace, open('/tmp/'+timestamp+'_'+'full_loss.pkl', 'wb')) 
    temp_path_scaler = 'file:/tmp/'+timestamp+'_'+'full_loss.pkl'
    final_path_scaler = 'dbfs:/FileStore/models/'+timestamp+'/full_loss.pkl'
    dbutils.fs.cp(temp_path_scaler, final_path_scaler)
      
    dump(full_vloss_trace, open('/tmp/'+timestamp+'_'+'full_vloss.pkl', 'wb')) 
    temp_path_scaler = 'file:/tmp/'+timestamp+'_'+'full_vloss.pkl'
    final_path_scaler = 'dbfs:/FileStore/models/'+timestamp+'/full_vloss.pkl'
    dbutils.fs.cp(temp_path_scaler, final_path_scaler)
  
  #print(np.array(vloss_trace).flatten())
  
  #print(full_loss_trace)
  #print(full_vloss_trace)


  #new_path = '/dbfs/FileStore/models/'+timestamp+'/'

  #plt.savefig('{}loss_{}.png'.format(new_path, save_name))  
  #plot_test(model,BoX_test,BoY_test,nvar=2,start=0,end=5000, name= 'loss.png') #) #.format(new_path, save_name))
  
  #def plot_test(model,x_test,y_test,nvar=2,name='test',start=0,end=500):
  start = 0
  end = 5000
  nvar = 2
  
  Y_predict = model.predict(BoX_test[start:end,:,:])
  #print(Y_predict.shape)
  fig, axs = plt.subplots(nvar,figsize=(14,12))
  
  for v in range (nvar):
      Y_test_var1 = BoY_test[start:end,v].reshape(-1,1)
      Y_predict_var1 = Y_predict[:,v].reshape(-1,1)
      axs[v].plot(Y_test_var1,label='Data')
      axs[v].plot(Y_predict_var1, label='Prediction')
        
  plt.savefig('predict_{}.png'.format(fold))
  mlflow.log_artifact('predict_{}.png'.format(fold))

In [0]:
def predict_plot(fold, model, BoX_test, BoY_test):
  import seaborn as sns
  #model=booster_model
  x_test=BoX_test
  y_test=BoY_test
  nvar=2
  name='test_diff'
  start=0
  end=1000
  x = np.linspace(start,end,int(end-start))
  Y_predict = model.predict(x_test[start:end,:,:])
  #print(Y_predict.shape)
  fig, axs = plt.subplots(nvar,figsize=(16,16))
  
  vnames = variables[0:nvar] #should be names
  
  for v in range(nvar):
      Y_test_var1 = unscale(str(vnames[v]), y_test[start:end,v].reshape(-1,1), scale_dict)
      Y_predict_var1 = unscale(str(vnames[v]), Y_predict[:,v].reshape(-1,1), scale_dict)
    
#       no more scaler
#       Y_test_var1 = data_list[v][0].inverse_transform(y_test[start:end,v].reshape(-1,1))
#       Y_predict_var1 = data_list[v][0].inverse_transform(Y_predict[:,v].reshape(-1,1))
      
      #axs[v].plot(Y_test_var1,Y_predict_var1,'o')
      mape = 100*abs(Y_test_var1-Y_predict_var1)/Y_test_var1
      #print(x.shape)
      #print(mape.shape)
      mape = mape.reshape(-1,)
      #print(mape.shape)
      #print('mape ave:{}'.format(mape.mean()))
      axs[v].plot(Y_test_var1,label='Data')
      axs[v].plot(Y_predict_var1, label='Digital Twin')
      #axs[v].fill_between(x, mape, -mape, color='red',edgecolor="black",alpha=0.5) 
      #axs[v].plot(mape)
      #axs[v].set_title(variables[v])
      axs[v].set_ylabel(variables[v])
      axs[v].set_xlabel('Time samples')
      axs[v].legend()

  #plt.savefig('/dbfs/FileStore/models/'+timestamp+'/'+mcp_name+'_prediction_final.png')
  plt.savefig("prediction_{}.png".format(fold))
  mlflow.log_artifact("prediction_{}.png".format(fold))

  # plt.savefig('{}.png'.format(name))

In [0]:
def data_distribution_plot(fold, model, BoX_test, BoY_test, scale_dict):
  import seaborn as sns
  fig, axs = plt.subplots(1,figsize=(12,12))
  x_test=BoX_test
  y_test=BoY_test
  start=0
  end=BoX_test.shape[0]
  Y_predict = model.predict(x_test[start:end,:,:])
  
  #indices from ['B:VIMIN', 'B:IMINER', 'B_VIMIN', 'B:LINFRQ', 'B:ACMNIG', 'B:ACMNPG', 'I:IB', 'I:MDAT40']

  # 0 is B:VIMIN
  # 1 is B:IMINER
  # 2 is B_VIMIN
  # 3 is B:LINFRQ
  # 4 is B:ACMNIG
  # 5 is B:ACMNPG
  # 6 is I:IB
  # 7 is I:MDAT40
  
  Y_test_var0 = unscale(str(variables[0]), y_test[start:end,0].reshape(-1,1), scale_dict).reshape(-1,1)
  Y_test_var1 = unscale(str(variables[1]), y_test[start:end,1].reshape(-1,1), scale_dict).reshape(-1,1)
  
  Y_predict_var0 = unscale(str(variables[0]), Y_predict[:,0].reshape(-1,1), scale_dict).reshape(-1,1)
  Y_predict_var1 = unscale(str(variables[1]), Y_predict[:,1].reshape(-1,1), scale_dict).reshape(-1,1)
  
  x_bvimin = unscale(str(variables[0]), x_test[start:end,0,-1].reshape(-1,1), scale_dict).reshape(-1,1)
  x_biminer = unscale(str(variables[1]), x_test[start:end,1,-1].reshape(-1,1), scale_dict).reshape(-1,1)
  x_b_vimin = unscale(str(variables[2]), x_test[start:end,2,-1].reshape(-1,1), scale_dict).reshape(-1,1)

#   Y_test_var0 = data_list[0][0].inverse_transform(y_test[start:end,0].reshape(-1,1)).reshape(-1,1)
#   Y_test_var1 = data_list[1][0].inverse_transform(y_test[start:end,1].reshape(-1,1)).reshape(-1,1)
  
#   Y_predict_var0 = data_list[0][0].inverse_transform(Y_predict[:,0].reshape(-1,1)).reshape(-1,1)
#   Y_predict_var1 = data_list[1][0].inverse_transform(Y_predict[:,1].reshape(-1,1)).reshape(-1,1)
  
#   x_bvimin = data_list[0][0].inverse_transform(x_test[start:end,0,-1].reshape(-1,1)).reshape(-1,1) # data B:VIMIN
#   x_biminer = data_list[1][0].inverse_transform(x_test[start:end,1,-1].reshape(-1,1)).reshape(-1,1) # data B:IMINER
#   x_b_vimin = data_list[2][0].inverse_transform(x_test[start:end,2,-1].reshape(-1,1)).reshape(-1,1) # data B_VIMIN

  #Rachael's/PID Eq
  
  # early March 2021, Kiyomi told me alpha =.01*B:ACMNPG and gamma = .00001*B:ACMNIG
  x_alpha = .01*unscale(str(variables[5]), x_test[start:end,5,-1].reshape(-1,1), scale_dict).reshape(-1,1)
  x_gamma = .00001*unscale(str(variables[4]), x_test[start:end,4,-1].reshape(-1,1), scale_dict).reshape(-1,1)
 
  alpha = x_alpha #10e-2
  gamma = x_gamma #7.535e-5
  beta=[0]
  for i in range(len(x_b_vimin)):
      if i>0:
          beta_t = beta[-1] + gamma[i]*x_biminer[i]
          beta.append(beta_t[0])
  
#   print(x_b_vimin.shape)
#   print(x_biminer.shape)
#   print(np.asarray(beta).reshape(-1,1).shape)
  
  beta = np.asarray(beta).reshape(-1,1)
  BVIMIN_rach = x_b_vimin -1*alpha*x_biminer -1*beta #predict the next, shiftting happens in the plotting
  print(BVIMIN_rach)

  BIMINER_rach = 10*np.add(x_b_vimin, -1*BVIMIN_rach)
#   print(BIMINER_rach)
#   print(x_biminer.shape)
#   print(x_b_vimin.shape)
#   print(BVIMIN_rach.shape)
#   print(BIMINER_rach.shape)
  
#   BVIMIN_rach = BVIMIN_rach.reshape(BVIMIN_rach.shape[0],1)
#   BIMINER_rach = BIMINER_rach.reshape(BIMINER_rach.shape[0],1)
  
#   print(Y_test_var0.shape)
#   print(Y_test_var1.shape)
#   print(Y_predict_var0.shape)
#   print(Y_predict_var1.shape)

  predicted = np.concatenate((Y_test_var0,Y_test_var1,Y_predict_var0,Y_predict_var1, BVIMIN_rach, BIMINER_rach),axis=concate_axis)
#   print(predicted.shape)
  df_cool = pd.DataFrame(predicted,columns=['data B:VIMIN','data B:IMINER','pred B:VIMIN','pred B:IMINER', 'PID B:VIMIN', 'PID B:IMINER'])
#   print(df_cool.shape)
  #sns.pairplot(df_predict)
  #np_data = np.concatenate((Y_predict_var0,Y_predict_var1),axis=concate_axis) 
  #df_data = pd.DataFrame(np_data)
  #sns.pairplot(df_data)
  sns.scatterplot(data=df_cool, x="data B:VIMIN", y="data B:IMINER", label='Data') #, hue="time")
  sns.scatterplot(data=df_cool, x="pred B:VIMIN", y="pred B:IMINER", label='Digital Twin') #, hue="time")
  sns.scatterplot(data=df_cool, x="PID B:VIMIN", y="PID B:IMINER", label="PID Eq")

  #sns.scatterplot(data=df_cool, x="data_va1", y="pred_va1", label='Data')#, hue="time")

  #axs.plot(Y_test_var0,Y_test_var1,'o', label='Data')
  #axs.plot(Y_predict_var0,Y_predict_var1,'*', label='Digital Twin')
  #axs.set_xlabel('B:VIMIN')
  #axs.set_ylabel('B:IMINER')
  #axs.set_xlim(103.2,103.6)
  
  #plt.savefig('/dbfs/FileStore/models/'+timestamp+'/'+mcp_name+'_corr_final.png')
  plt.savefig("corr_{}.png".format(fold))
  mlflow.log_artifact("corr_{}.png".format(fold))
  
  #plt.legend()

In [0]:
def statespace_distribution_plot(fold, model, BoX_test, BoY_test):
  import seaborn as sns
  fig, axs = plt.subplots(1,figsize=(12,12))
  x_test=BoX_test
  y_test=BoY_test
  start=0
  end=BoX_test.shape[0]
  Y_predict = model.predict(x_test[start:end,:,:])
  
  Y_test_var0 = unscale(str(variables[0]), y_test[start:end,0].reshape(-1,1), scale_dict).reshape(-1,1)
  Y_test_var1 = unscale(str(variables[1]), y_test[start:end,1].reshape(-1,1), scale_dict).reshape(-1,1)
  
  Y_predict_var0 = unscale(str(variables[0]), Y_predict[:,0].reshape(-1,1), scale_dict).reshape(-1,1)
  Y_predict_var1 = unscale(str(variables[1]), Y_predict[:,1].reshape(-1,1), scale_dict).reshape(-1,1)
  
  x_bvimin = unscale(str(variables[0]), x_test[start:end,0,-1].reshape(-1,1), scale_dict).reshape(-1,1)
  x_biminer = unscale(str(variables[1]), x_test[start:end,1,-1].reshape(-1,1), scale_dict).reshape(-1,1)
  x_b_vimin = unscale(str(variables[2]), x_test[start:end,2,-1].reshape(-1,1), scale_dict).reshape(-1,1)
  
#   Y_test_var0 = data_list[0][0].inverse_transform(y_test[start:end,0].reshape(-1,1)).reshape(-1,1)
#   Y_test_var1 = data_list[1][0].inverse_transform(y_test[start:end,1].reshape(-1,1)).reshape(-1,1)
  
#   Y_predict_var0 = data_list[0][0].inverse_transform(Y_predict[:,0].reshape(-1,1)).reshape(-1,1)
#   Y_predict_var1 = data_list[1][0].inverse_transform(Y_predict[:,1].reshape(-1,1)).reshape(-1,1)
  
#   x_bvimin = data_list[0][0].inverse_transform(x_test[start:end,0,-1].reshape(-1,1)).reshape(-1,1) # data B:VIMIN
#   x_biminer = data_list[1][0].inverse_transform(x_test[start:end,1,-1].reshape(-1,1)).reshape(-1,1) # data B:IMINER
#   x_b_vimin = data_list[2][0].inverse_transform(x_test[start:end,2,-1].reshape(-1,1)).reshape(-1,1) # data B_VIMIN

# early March 2021, Kiyomi told me alpha =.01*B:ACMNPG and gamma = .00001*B:ACMNIG
  x_alpha = .01*unscale(str(variables[5]), x_test[start:end,5,-1].reshape(-1,1), scale_dict).reshape(-1,1)
  x_gamma = .00001*unscale(str(variables[4]), x_test[start:end,4,-1].reshape(-1,1), scale_dict).reshape(-1,1)
 
  alpha = x_alpha #10e-2
  gamma = x_gamma #7.535e-5
  beta=[0]
  for i in range(len(x_b_vimin)):
      if i>0:
          beta_t = beta[-1] + gamma[i]*x_biminer[i]
          beta.append(beta_t[0])
  
#   print(x_b_vimin.shape)
#   print(x_biminer.shape)
#   print(np.asarray(beta).reshape(-1,1).shape)
  
  beta = np.asarray(beta).reshape(-1,1)
  BVIMIN_rach = x_b_vimin -1*alpha*x_biminer -1*beta #predict the next, shiftting happens in the plotting
  BIMINER_rach = 10*np.add(x_b_vimin, -1*BVIMIN_rach)
  
# #   print(BIMINER_rach)
# #   print(x_biminer.shape)
# #   print(x_b_vimin.shape)
# #   print(BVIMIN_rach.shape)
# #   print(BIMINER_rach.shape)
  
# #   BVIMIN_rach = BVIMIN_rach.reshape(BVIMIN_rach.shape[0],1)
# #   BIMINER_rach = BIMINER_rach.reshape(BIMINER_rach.shape[0],1)
  
# #   print(Y_test_var0.shape)
# #   print(Y_test_var1.shape)
# #   print(Y_predict_var0.shape)
# #   print(Y_predict_var1.shape)

  predicted = np.concatenate((Y_test_var0,Y_test_var1,Y_predict_var0,Y_predict_var1, BVIMIN_rach, BIMINER_rach),axis=concate_axis)
#   print(predicted.shape)
  df_cool = pd.DataFrame(predicted,columns=['data B:VIMIN','data B:IMINER','pred B:VIMIN','pred B:IMINER', 'PID B:VIMIN', 'PID B:IMINER'])
  df_cool = df_cool.diff()
#   print(df_cool.diff())
  #sns.pairplot(df_predict)
  #np_data = np.concatenate((Y_predict_var0,Y_predict_var1),axis=concate_axis) 
  #df_data = pd.DataFrame(np_data)
  #sns.pairplot(df_data)
  sns.scatterplot(data=df_cool, x="data B:VIMIN", y="data B:IMINER", label='Data Diff') #, hue="time")
  sns.scatterplot(data=df_cool, x="pred B:VIMIN", y="pred B:IMINER", label='Digital Twin Diff') #, hue="time")
  sns.scatterplot(data=df_cool, x="PID B:VIMIN", y="PID B:IMINER", label="PID Eq Diff")

  #sns.scatterplot(data=df_cool, x="data_va1", y="pred_va1", label='Data')#, hue="time")

  #axs.plot(Y_test_var0,Y_test_var1,'o', label='Data')
  #axs.plot(Y_predict_var0,Y_predict_var1,'*', label='Digital Twin')
  #axs.set_xlabel('B:VIMIN')
  #axs.set_ylabel('B:IMINER')
  #axs.set_xlim(103.2,103.6)
  
  #plt.savefig('/dbfs/FileStore/models/'+timestamp+'/'+mcp_name+'_corr_final.png')
  plt.savefig("statespace_corr_{}.png".format(fold))
  mlflow.log_artifact("statespace_corr_{}.png".format(fold))
  
  #plt.legend()

In [0]:
def data_heatmap(fold, model, BoX_test, BoY_test):

  import matplotlib.pyplot as plt
  from mpl_toolkits.axes_grid1 import make_axes_locatable
  import numpy as np

  x_test=BoX_test
  y_test=BoY_test
  start=0
  end=BoX_test.shape[0]
  Y_predict = model.predict(x_test[start:end,:,:])
  
  Y_test_var0 = unscale(str(variables[0]), y_test[start:end,0].reshape(-1,1), scale_dict).reshape(-1,1)
  Y_test_var1 = unscale(str(variables[1]), y_test[start:end,1].reshape(-1,1), scale_dict).reshape(-1,1)
  
  Y_predict_var0 = unscale(str(variables[0]), Y_predict[:,0].reshape(-1,1), scale_dict).reshape(-1,1)
  Y_predict_var1 = unscale(str(variables[1]), Y_predict[:,1].reshape(-1,1), scale_dict).reshape(-1,1)
  
  x_bvimin = unscale(str(variables[0]), x_test[start:end,0,-1].reshape(-1,1), scale_dict).reshape(-1,1)
  x_biminer = unscale(str(variables[1]), x_test[start:end,1,-1].reshape(-1,1), scale_dict).reshape(-1,1)
  x_b_vimin = unscale(str(variables[2]), x_test[start:end,2,-1].reshape(-1,1), scale_dict).reshape(-1,1)
  
#   Y_test_var0 = data_list[0][0].inverse_transform(y_test[start:end,0].reshape(-1,1)).reshape(-1,1)
#   Y_test_var1 = data_list[1][0].inverse_transform(y_test[start:end,1].reshape(-1,1)).reshape(-1,1)

#   Y_predict_var0 = data_list[0][0].inverse_transform(Y_predict[:,0].reshape(-1,1)).reshape(-1,1)
#   Y_predict_var1 = data_list[1][0].inverse_transform(Y_predict[:,1].reshape(-1,1)).reshape(-1,1)

#   x_bvimin = data_list[0][0].inverse_transform(x_test[start:end,0,-1].reshape(-1,1)).reshape(-1,1) # data B:VIMIN
#   x_biminer = data_list[1][0].inverse_transform(x_test[start:end,1,-1].reshape(-1,1)).reshape(-1,1) # data B:IMINER
#   x_b_vimin = data_list[2][0].inverse_transform(x_test[start:end,2,-1].reshape(-1,1)).reshape(-1,1) # data B_VIMIN

  # early March 2021, Kiyomi told me alpha =.01*B:ACMNPG and gamma = .00001*B:ACMNIG
  x_alpha = .01*unscale(str(variables[5]), x_test[start:end,5,-1].reshape(-1,1), scale_dict).reshape(-1,1)
  x_gamma = .00001*unscale(str(variables[4]), x_test[start:end,4,-1].reshape(-1,1), scale_dict).reshape(-1,1)
 
  alpha = x_alpha #10e-2
  gamma = x_gamma #7.535e-5
  beta=[0]
  for i in range(len(x_b_vimin)):
      if i>0:
          beta_t = beta[-1] + gamma[i]*x_biminer[i]
          beta.append(beta_t[0])

  #   print(x_b_vimin.shape)
  #   print(x_biminer.shape)
  #   print(np.asarray(beta).reshape(-1,1).shape)

  beta = np.asarray(beta).reshape(-1,1)
  BVIMIN_rach = x_b_vimin -1*alpha*x_biminer -1*beta #predict the next, shiftting happens in the plotting
  BIMINER_rach = 10*np.add(x_b_vimin, -1*BVIMIN_rach)

  #   print(BVIMIN_rach)

  #print(Y_test_var0)
  #print(BIMINER_rach)

  predicted = np.concatenate((Y_test_var0,Y_test_var1,Y_predict_var0,Y_predict_var1, BVIMIN_rach, BIMINER_rach),axis=concate_axis)
  df_cool = pd.DataFrame(predicted,columns=['data B:VIMIN','data B:IMINER','pred B:VIMIN','pred B:IMINER', 'PID B:VIMIN', 'PID B:IMINER'])

  #df_cool.head(10)
  # #x is B:VIMIN
  # #y is B:IMINER

  # # Generate some test data
  x = np.asarray(df_cool['data B:VIMIN'])
  y = np.asarray(df_cool['data B:IMINER'])
  heatmap, xedges, yedges = np.histogram2d(x, y, bins=[200,200], density=True, range = [[103.2, 103.6], [-.6, .6]])
  
#   print(min(x), max(x))

  #print(xedges, yedges)
  #extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

  # #print(extent)

  x2 = np.asarray(df_cool['pred B:VIMIN'])
  y2 = np.asarray(df_cool['pred B:IMINER'])
  #   print(x2.shape)
  #   print(y2.shape)
  heatmap2, xedges2, yedges2 = np.histogram2d(x2, y2, bins=[200,200], density=True, range = [[103.2, 103.6], [-1, 1]])
  # #extent2 = [xedges2[0], xedges2[-1], yedges2[0], yedges2[-1]]

  #print(xedges2, yedges2)
  # #print(extent2)

  x3 = np.asarray(df_cool['PID B:VIMIN']) #.reshape(-1, )
  y3 = np.asarray(df_cool['PID B:IMINER']) #.reshape(-1, )
  #   print(x3.shape)
  #   print(y3.shape)
  heatmap3, xedges3, yedges3 = np.histogram2d(x3, y3, bins=[200,200], density=True, range = [[103.2, 103.6], [-1, 1]])
  #extent3 = [xedges3[0], xedges3[-1], yedges3[0], yedges3[-1]]

  #print(xedges3, yedges3)
  #extent = [min(x), max(x)]

  #sns.scatterplot(data=df_cool, x="data B:VIMIN", y="data B:IMINER", label='Data') #, hue="time")

  fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(15, 24))
#   fig.tight_layout()
  ax[0].set_title('Data')
  a = ax[0].imshow(heatmap, interpolation='nearest', origin='lower', aspect='auto')
  ax[1].set_title('Digital Twin')
  b = ax[1].imshow(heatmap2, interpolation='nearest',origin='lower', aspect='auto')
  ax[2].set_title('PID Eq')
  c = ax[2].imshow(heatmap3, interpolation='nearest',origin='lower', aspect='auto')
  #fig.subplots_adjust(hspace=-.05)
  
  i = 0
  p = [a, b, c]
  for ax in fig.get_axes():
    ax.set_ylabel('B:IMINER')
    ax.set_xlabel('B:VIMIN')  
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", "5%", pad="3%")
    ax.figure.colorbar(p[i], cax=cax)
    i+=1

  plt.savefig("heatmap_{}.png".format(fold))
  mlflow.log_artifact("heatmap_{}.png".format(fold))
  #   plt.show()

  #need to add colorbars too
# data_heatmap(1, booster_model, BoX_test, BoY_test)